In this notebook, we will cover more topics related to regular expressions:

* [Repetition](#Repetition)
* [Grouping](#Grouping)
* [Detect matches](#Detect-matches)
* [Extract matches](#Extract-matches)

In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Repetition

We can control how many times a pattern matches:

* `?`: 0 or 1
* `+`: 1 or more
* `*`: 0 or more

In [2]:
x <- c("cat", "dog", "dogs", "cats")
str_extract(x, "cats?")

[1] "cat"  NA     NA     "cats"

In [3]:
str_extract_all("Why? No really, why?", "\\?") # to match a literal question mark

[[1]]
[1] "?" "?"

In [4]:
x <- c("hiking is fun", "reading is fun", "driving is not fun", "flying is not fun", "biking is fun")
str_extract(x, "is (not )?fun")

[1] "is fun"     "is fun"     "is not fun" "is not fun" "is fun"

In [5]:
x <- c("abc", "bc", "aabc", "aaabc")
str_extract(x, "a+")

[1] "a"   NA    "aa"  "aaa"

In [6]:
x <- c("ac", "abc", "abbc", "xyz", "abbbbc")
str_extract(x, "ab*c")

[1] "ac"     "abc"    "abbc"   NA       "abbbbc"

We can also specify the number of matches precisely:

* `{n}`: exactly n
* `{n,}`: n or more
* `{n,m}`: between n and m

In [7]:
str_extract(x, "ab{2}c")

[1] NA     NA     "abbc" NA     NA

In [8]:
str_extract(x, "ab{2,}c")

[1] NA       NA       "abbc"   NA       "abbbbc"

In [9]:
str_extract(x, "ab{0,2}c")

[1] "ac"   "abc"  "abbc" NA     NA

# Grouping

Parentheses define groups that can be referred to as `\1`, `\2` etc.

In [10]:
x <- c("mom", "dad", "brother", "sister")
re <- "^(.).*\\1$" 
str_extract(x, re) # find strings that start and end with the same character

[1] "mom" "dad" NA    NA

In [11]:
x <- c("he moved his head", "she moved her car", "nobody moved anything", "they moved their bikes")
re <- "(..).*\\1" # find a repeated pair of characters
str_extract(x, re)

[1] "he moved his he" "he moved he"     NA                "they moved th"

# Detect matches

In [12]:
x <- c("x-ray", "something", "xylophone", "xenophobia", "nothing")
str_detect(x, "^x") # which strings start with x?

[1]  TRUE FALSE  TRUE  TRUE FALSE

In [13]:
head(words, 10) # words is a predefined character vector 

[1] "a"        "able"     "about"    "absolute" "accept"   "account" 
 [7] "achieve"  "across"   "act"      "active"

In [14]:
sum(str_detect(words, "^a")) # no. of words that start with a

[1] 65

In [15]:
# for every letter, find how many words start with it
for (let in letters) {
    print(sprintf("%d words start with %s", sum(str_detect(words, str_c("^", let))), let))
}

[1] "65 words start with a"
[1] "58 words start with b"
[1] "83 words start with c"
[1] "43 words start with d"
[1] "45 words start with e"
[1] "54 words start with f"
[1] "23 words start with g"
[1] "38 words start with h"
[1] "25 words start with i"
[1] "6 words start with j"
[1] "9 words start with k"
[1] "45 words start with l"
[1] "45 words start with m"
[1] "23 words start with n"
[1] "28 words start with o"
[1] "72 words start with p"
[1] "7 words start with q"
[1] "46 words start with r"
[1] "119 words start with s"
[1] "65 words start with t"
[1] "12 words start with u"
[1] "8 words start with v"
[1] "53 words start with w"
[1] "0 words start with x"
[1] "6 words start with y"
[1] "0 words start with z"


In [16]:
sum(str_detect(words, "^[^aeiou]*$")) # no. of words with no vowels

[1] 6

In [17]:
words[str_detect(words, "^[^aeiou]*$")] # logical subsetting to get the words themselves

[1] "by"  "dry" "fly" "mrs" "try" "why"

In [18]:
str_subset(words, "^[^aeiou]*$") # str_subset is a convenient shortcut

[1] "by"  "dry" "fly" "mrs" "try" "why"

In [19]:
str_subset(words, "^[aeiou]*$") # words with only vowels

[1] "a"

In [20]:
df <- tibble(
  word = words, 
  i = seq_along(word)
)
print(df)

# A tibble: 980 × 2
   word         i
   <chr>    <int>
 1 a            1
 2 able         2
 3 about        3
 4 absolute     4
 5 accept       5
 6 account      6
 7 achieve      7
 8 across       8
 9 act          9
10 active      10
# … with 970 more rows


In [21]:
df %>%
    filter(str_detect(word, "^q")) # can use str_detect with the dplyr verb filter

word,i
<chr>,<int>
quality,665
quarter,666
question,667
quick,668
quid,669
quiet,670
quite,671


In [22]:
str_extract_all("xyxyxyx", "xyx") # not how matches don't overlap

[[1]]
[1] "xyx" "xyx"

In [23]:
str_count("xyxyxyx", "xyx") # gives the number of matches

[1] 2

In [24]:
df <- df %>% # mutate with str_count
            mutate(num_vowels = str_count(word, "[aeiou]"),
                   num_consonants = str_count(word, "[^aeiou]"))
print(df)

# A tibble: 980 × 4
   word         i num_vowels num_consonants
   <chr>    <int>      <int>          <int>
 1 a            1          1              0
 2 able         2          2              2
 3 about        3          3              2
 4 absolute     4          4              4
 5 accept       5          2              4
 6 account      6          3              4
 7 achieve      7          4              3
 8 across       8          2              4
 9 act          9          1              2
10 active      10          3              3
# … with 970 more rows


In [25]:
df <- df %>% # compute a new column having proportion of vowels in the words
            mutate(vowel_prop = num_vowels / str_length(word)) 
print(df)

# A tibble: 980 × 5
   word         i num_vowels num_consonants vowel_prop
   <chr>    <int>      <int>          <int>      <dbl>
 1 a            1          1              0      1    
 2 able         2          2              2      0.5  
 3 about        3          3              2      0.6  
 4 absolute     4          4              4      0.5  
 5 accept       5          2              4      0.333
 6 account      6          3              4      0.429
 7 achieve      7          4              3      0.571
 8 across       8          2              4      0.333
 9 act          9          1              2      0.333
10 active      10          3              3      0.5  
# … with 970 more rows


In [26]:
filter(df, str_detect(word, "a"), str_detect(word, "e"), str_detect(word, "i"),
       str_detect(word, "o"), str_detect(word, "u")) # words containing all vowels

word,i,num_vowels,num_consonants,vowel_prop
<chr>,<int>,<int>,<int>,<dbl>


In [27]:
# words with at least 4 different vowels
filter(df, str_detect(word, "a") + str_detect(word, "e") + str_detect(word, "i") +
       str_detect(word, "o") + str_detect(word, "u") >= 4)

word,i,num_vowels,num_consonants,vowel_prop
<chr>,<int>,<int>,<int>,<dbl>
absolute,4,4,4,0.5000000
appropriate,48,5,6,0.4545455
associate,57,5,4,0.5555556
authority,61,4,5,0.4444444
colleague,166,5,4,0.5555556
continue,186,4,4,0.5000000
encourage,268,5,4,0.5555556
introduce,431,4,5,0.4444444
organize,585,4,4,0.5000000


# Extract matches

In [28]:
v <- "[aeiou]"
c <- "[^aeiou]"
vvcv <- str_c(v, v, c, v)
print(vvcv)

[1] "[aeiou][aeiou][^aeiou][aeiou]"


In [29]:
(vvcv_words <- str_subset(words, vvcv)) # words containing the pattern vvcv

[1] "achieve"     "appropriate" "associate"   "available"   "because"    
 [6] "believe"     "cause"       "choice"      "choose"      "colleague"  
[11] "create"      "encourage"   "europe"      "house"       "increase"   
[16] "language"    "leave"       "meaning"     "measure"     "piece"      
[21] "please"      "quality"     "quite"       "raise"       "realise"    
[26] "reason"      "receive"     "require"     "situate"     "square"     
[31] "thousand"

In [30]:
str_extract(vvcv_words, vvcv) # the patterns themselves

[1] "ieve" "iate" "iate" "aila" "ause" "ieve" "ause" "oice" "oose" "eagu"
[11] "eate" "oura" "euro" "ouse" "ease" "uage" "eave" "eani" "easu" "iece"
[21] "ease" "uali" "uite" "aise" "eali" "easo" "eive" "uire" "uate" "uare"
[31] "ousa"

In [31]:
cvvc <- str_c(c, v, v, c)
print(cvvc)

[1] "[^aeiou][aeiou][aeiou][^aeiou]"


In [32]:
(cvvc_words <- str_subset(words, cvvc)) # words containing the pattern cvvc

[1] "about"       "account"     "achieve"     "actual"      "afternoon"  
  [6] "again"       "against"     "already"     "although"    "amount"     
 [11] "appear"      "appoint"     "approach"    "appropriate" "around"     
 [16] "associate"   "available"   "bear"        "beat"        "because"    
 [21] "believe"     "between"     "blood"       "board"       "boat"       
 [26] "book"        "break"       "brief"       "brilliant"   "britain"    
 [31] "build"       "cause"       "certain"     "chair"       "chairman"   
 [36] "cheap"       "choice"      "choose"      "claim"       "clean"      
 [41] "clear"       "client"      "colleague"   "colour"      "condition"  
 [46] "cook"        "could"       "council"     "count"       "country"    
 [51] "county"      "couple"      "course"      "court"       "create"     
 [56] "dead"        "deal"        "dear"        "decision"    "deep"       
 [61] "detail"      "door"        "double"      "doubt"       "encourage"  
 [66] "enough"      "equal"       "especial"    "experience"  "explain"    
 [71] "fair"        "favour"      "feed"        "feel"        "field"      
 [76] "floor"       "food"        "foot"        "four"        "friend"     
 [81] "function"    "good"        "goodbye"     "great"       "green"      
 [86] "ground"      "group"       "guess"       "hair"        "head"       
 [91] "health"      "hear"        "heart"       "heat"        "heavy"      
 [96] "hour"        "house"       "increase"    "indeed"      "individual" 
[101] "instead"     "join"        "keep"        "labour"      "language"   
[106] "laugh"       "lead"        "learn"       "leave"       "load"       
[111] "look"        "main"        "mean"        "meaning"     "measure"    
[116] "meet"        "mention"     "million"     "motion"      "nation"     
[121] "near"        "need"        "occasion"    "paint"       "pair"       
[126] "pension"     "people"      "period"      "piece"       "please"     
[131] "point"       "poor"        "position"    "pound"       "proceed"    
[136] "quality"     "quarter"     "question"    "quick"       "quid"       
[141] "quite"       "rail"        "raise"       "read"        "ready"      
[146] "real"        "realise"     "really"      "reason"      "receive"    
[151] "region"      "relation"    "require"     "research"    "resource"   
[156] "road"        "room"        "round"       "school"      "science"    
[161] "seat"        "section"     "seem"        "sheet"       "shoot"      
[166] "should"      "situate"     "sleep"       "social"      "society"    
[171] "soon"        "sound"       "south"       "speak"       "special"    
[176] "speed"       "square"      "stairs"      "station"     "straight"   
[181] "street"      "succeed"     "suit"        "teach"       "team"       
[186] "television"  "thirteen"    "though"      "thousand"    "through"    
[191] "touch"       "train"       "treat"       "trouble"     "tuesday"    
[196] "union"       "usual"       "view"        "wait"        "wear"       
[201] "week"        "weigh"       "without"     "wood"        "would"      
[206] "year"        "young"

In [33]:
str_extract(cvvc_words, cvvc) # the patterns themselves

[1] "bout" "coun" "hiev" "tual" "noon" "gain" "gain" "read" "houg" "moun"
 [11] "pear" "poin" "roac" "riat" "roun" "ciat" "vail" "bear" "beat" "caus"
 [21] "liev" "ween" "lood" "boar" "boat" "book" "reak" "rief" "lian" "tain"
 [31] "buil" "caus" "tain" "hair" "hair" "heap" "hoic" "hoos" "laim" "lean"
 [41] "lear" "lien" "leag" "lour" "tion" "cook" "coul" "coun" "coun" "coun"
 [51] "coun" "coup" "cour" "cour" "reat" "dead" "deal" "dear" "sion" "deep"
 [61] "tail" "door" "doub" "doub" "cour" "noug" "qual" "cial" "rien" "lain"
 [71] "fair" "vour" "feed" "feel" "fiel" "loor" "food" "foot" "four" "rien"
 [81] "tion" "good" "good" "reat" "reen" "roun" "roup" "gues" "hair" "head"
 [91] "heal" "hear" "hear" "heat" "heav" "hour" "hous" "reas" "deed" "dual"
[101] "tead" "join" "keep" "bour" "guag" "laug" "lead" "lear" "leav" "load"
[111] "look" "main" "mean" "mean" "meas" "meet" "tion" "lion" "tion" "tion"
[121] "near" "need" "sion" "pain" "pair" "sion" "peop" "riod" "piec" "leas"
[131] "poin" "poor" "tion" "poun" "ceed" "qual" "quar" "ques" "quic" "quid"
[141] "quit" "rail" "rais" "read" "read" "real" "real" "real" "reas" "ceiv"
[151] "gion" "tion" "quir" "sear" "sour" "road" "room" "roun" "hool" "cien"
[161] "seat" "tion" "seem" "heet" "hoot" "houl" "tuat" "leep" "cial" "ciet"
[171] "soon" "soun" "sout" "peak" "cial" "peed" "quar" "tair" "tion" "raig"
[181] "reet" "ceed" "suit" "teac" "team" "sion" "teen" "houg" "hous" "roug"
[191] "touc" "rain" "reat" "roub" "tues" "nion" "sual" "view" "wait" "wear"
[201] "week" "weig" "hout" "wood" "woul" "year" "youn"

In [34]:
(q_words <- str_subset(words, "^q"))

[1] "quality"  "quarter"  "question" "quick"    "quid"     "quiet"    "quite"

In [35]:
str_extract(q_words, v) # extracts only the first match

[1] "u" "u" "u" "u" "u" "u" "u"

In [36]:
str_extract_all(q_words, v) # extracts all matches

[[1]]
[1] "u" "a" "i"

[[2]]
[1] "u" "a" "e"

[[3]]
[1] "u" "e" "i" "o"

[[4]]
[1] "u" "i"

[[5]]
[1] "u" "i"

[[6]]
[1] "u" "i" "e"

[[7]]
[1] "u" "i" "e"

Setting `simplify = TRUE` returns a matrix with shorter matches expanded to match the length of the longest

In [37]:
str_extract_all(q_words, v, simplify = TRUE)

u,a,i,
u,a,e,
u,e,i,o
u,i,,
u,i,,
u,i,e,
u,i,e,
